# CSP

In [422]:
import re
import numpy as np
import time

In [423]:
with open('input.txt','r') as f:
    b = f.readlines()

In [424]:
b = b[2:]

## Array

In [425]:
# returns the line number where matrix ends

def string_matrix_index(text):
    for i,row in enumerate(text):
        if  row[0]=='#':
            return i-1

In [426]:
matrix_idx = string_matrix_index(b)

In [427]:
string_matrix = b[0:matrix_idx]

In [428]:
# more comprehensive but slower version of below one-liner

# def get_matrix(s):
#     res = []
#     for k in s:
#         b1 = [int(x) if x.isdigit() else 0 if i%2==0 else -1 for i,x in enumerate(k)]
#         bint = [i for i in b1 if i!=-1]
#         res.append(bint)
#     return np.array(res)

In [429]:
# get matrix
arr = [[i for i in [int(x) if x.isdigit() else 0 if i%2==0 else -1 for i,x in enumerate(k)] if i!=-1] for k in string_matrix]

In [430]:
arr = np.array(arr)[:,:-1]

In [431]:
# please check that the array shape is 20x20, otherwise there's problem with the input
arr.shape

(20, 20)

In [432]:
arr

array([[2, 2, 1, 3, 0, 3, 4, 0, 4, 2, 2, 4, 0, 4, 0, 0, 0, 2, 4, 3],
       [0, 2, 1, 3, 2, 2, 0, 0, 0, 1, 0, 1, 2, 2, 0, 2, 3, 4, 4, 4],
       [2, 1, 0, 0, 0, 2, 2, 4, 3, 2, 2, 2, 1, 2, 2, 0, 0, 1, 1, 0],
       [0, 3, 2, 0, 1, 0, 1, 1, 4, 1, 3, 4, 2, 1, 0, 0, 0, 1, 4, 1],
       [4, 2, 3, 4, 4, 4, 0, 2, 2, 0, 0, 3, 2, 3, 0, 2, 0, 2, 3, 2],
       [2, 4, 1, 0, 1, 1, 1, 4, 1, 0, 1, 3, 0, 3, 2, 0, 4, 0, 1, 2],
       [4, 3, 1, 4, 3, 3, 1, 4, 0, 1, 1, 1, 4, 2, 2, 4, 1, 4, 3, 4],
       [2, 1, 1, 1, 4, 4, 3, 4, 4, 2, 0, 3, 0, 3, 0, 3, 4, 4, 0, 0],
       [1, 0, 0, 3, 0, 3, 2, 2, 4, 3, 3, 2, 4, 0, 0, 0, 1, 4, 3, 0],
       [0, 1, 3, 1, 3, 2, 3, 2, 1, 0, 0, 1, 4, 3, 2, 1, 2, 4, 1, 3],
       [0, 3, 0, 2, 3, 4, 3, 3, 1, 0, 0, 1, 2, 3, 3, 0, 2, 1, 2, 3],
       [1, 3, 1, 1, 4, 4, 4, 2, 0, 4, 4, 3, 4, 1, 1, 2, 2, 3, 2, 3],
       [2, 1, 0, 1, 1, 1, 0, 1, 2, 1, 3, 2, 2, 3, 3, 4, 0, 0, 3, 4],
       [4, 2, 0, 0, 2, 3, 2, 1, 1, 1, 4, 0, 4, 4, 3, 2, 2, 0, 2, 3],
       [3, 1, 4, 3, 0, 1, 4, 4, 1,

## Targets

In [433]:
targets = b[matrix_idx+5:matrix_idx+9]

In [434]:
targets = [int(target.strip()[2:]) for target in targets ]

In [435]:
targets

[18, 19, 16, 17]

## Tiles

In [436]:
tiles = b[matrix_idx+2:matrix_idx+3][0].strip()

In [437]:
shapes = {}

In [438]:
shapes['OUTER_BOUNDARY'] = int(re.split(',|}',re.compile("OUTER_BOUNDARY=").split(tiles)[1])[0])

In [439]:
shapes['EL_SHAPE'] = int(re.split(',|}',re.compile("EL_SHAPE=").split(tiles)[1])[0])

In [440]:
shapes['FULL_BLOCK'] = int(re.split(',|}',re.compile("FULL_BLOCK=").split(tiles)[1])[0])

In [441]:
shapes

{'OUTER_BOUNDARY': 6, 'EL_SHAPE': 7, 'FULL_BLOCK': 12}

## Count 

In [442]:
# returns count of visible bushes in 4x4 area after certain tile is placed 

def calc_bush(arr,tile):
    if tile=='OUTER_BOUNDARY':
        return dict(zip(*np.unique(arr[1:3,1:3], return_counts=True)))
    if tile=='EL_SHAPE':
        return dict(zip(*np.unique(arr[1:,1:], return_counts=True)))
    return {1:0,2:0,3:0,4:0}

In [443]:
# initializes states

def fill_states(n):
    domain = {}
    variables = {}
    for row in range(n):
        for col in range(n):
            domain[(row,col)] = ['OUTER_BOUNDARY','EL_SHAPE','FULL_BLOCK']
            variables[(row,col)] = 0
    return domain,variables

In [444]:
# adds bush count in 4x4 area to overall bush count

def add_bush(state,d):
    count_dict = calc_bush(arr[4*state[0]:4*state[0]+4,4*state[1]:4*state[1]+4],d)
    for item in count_dict.items():
        if item[0]!=0:
            bushes[item[0]-1]+=item[1] 

In [445]:
# removes bush count in 4x4 area from overall bush count

def remove_bush(state,d):
    count_dict = calc_bush(arr[4*state[0]:4*state[0]+4,4*state[1]:4*state[1]+4],d)
    for item in count_dict.items():
        if item[0]!=0:
            bushes[item[0]-1]-=item[1] 

In [446]:
# checks if the constraints are satisfied

def check_constraints(state,d,tile_count):
    add_bush(state,d)    
    for i in range(4):
        if bushes[i]>targets[i]:
            remove_bush(state,d)
            return False
    if tile_count[d]+1>shapes[d]:
            remove_bush(state,d)
            return False
    return True

In [447]:
# calculate number of non-zero values for every square
# will help choosing the next square to put a tile on

def calc_number_count(n):
    variables = {}
    for row in range(n):
        for col in range(n):
            variables[(row,col)] = np.count_nonzero(arr[4*row:4*row+4,4*col:4*col+4]) 
    return variables

In [448]:
# the first method used to get next state (not used in current version)

def check_full():
    for state in states:
        if states[state]==0:
            return state
    return (-1,-1)

In [449]:
# gets next state based on mrv

def get_next_state():
    for state in square_order:
        if states[state]==0:
            return state
    return (-1,-1)

In [450]:
bushes = [0,0,0,0]
tile_count = {'OUTER_BOUNDARY':0,'EL_SHAPE':0,'FULL_BLOCK':0}
domain,states = fill_states(len(arr)//4)
num_count = calc_number_count(len(arr)//4)

# sorting squares based on non-zero number count in descending order
square_order = [k for k,v in sorted(num_count.items(), key=lambda x: x[1])]

def search(verbose=False):
    # gets the next state to be checked
    state = get_next_state()
    
    # if we have reached the goal return
    if bushes == targets and tile_count==shapes:
        return True
    
    # check possible tiles and add them if they satisfy the constraint
    for d in ['OUTER_BOUNDARY', 'EL_SHAPE', 'FULL_BLOCK']:
        if check_constraints(state,d,tile_count):
            states[state] = d
            tile_count[states[state]]+=1
            if verbose==True:
                print('state: ',state,' value: ',states[state],' tile count: ',tile_count, ' bushes: ',bushes)
            if search(verbose=verbose):
                return True
            
            # remove current tile to backtrack
            remove_bush(state,states[state])
            tile_count[states[state]]-=1
            states[state] = 0
            

    return False

In [451]:
dblStart = time.perf_counter()
search()
intTime = "%.2f" % ((time.perf_counter() - dblStart)) 
print("Elasped Time: " + str(intTime) + " seconds", "\n")

Elasped Time: 0.38 seconds 



In [452]:
for i,state in enumerate(states):
    print(i,4,states[state])

0 4 OUTER_BOUNDARY
1 4 OUTER_BOUNDARY
2 4 FULL_BLOCK
3 4 OUTER_BOUNDARY
4 4 FULL_BLOCK
5 4 FULL_BLOCK
6 4 EL_SHAPE
7 4 OUTER_BOUNDARY
8 4 OUTER_BOUNDARY
9 4 EL_SHAPE
10 4 OUTER_BOUNDARY
11 4 FULL_BLOCK
12 4 EL_SHAPE
13 4 EL_SHAPE
14 4 FULL_BLOCK
15 4 FULL_BLOCK
16 4 FULL_BLOCK
17 4 FULL_BLOCK
18 4 FULL_BLOCK
19 4 EL_SHAPE
20 4 EL_SHAPE
21 4 FULL_BLOCK
22 4 FULL_BLOCK
23 4 EL_SHAPE
24 4 FULL_BLOCK


In [453]:
# checking the correctness of the answer
bushes = [0,0,0,0]
for state in states:
    add_bush(state,states[state])
print(bushes==targets)

True
